In [57]:
from pandas_summary import DataFrameSummary
import redcap
import pandas as pd
import macos_keychain
import re
import os

In [17]:
# Avoid plain text password (Reference: https://gist.github.com/gcollazo/9434580)
def getpassword(service, account):

    def decode_hex(s):
        s = eval('"' + re.sub(r"(..)", r"\x\1", s) + '"')
        if "" in s: s = s[:s.index("")]
        return s

    cmd = ' '.join([
        "/usr/bin/security",
        " find-generic-password",
        "-g -s '%s' -a '%s'" % (service, account),
        "2>&1 >/dev/null"
    ])
    p = os.popen(cmd)
    s = p.read()
    p.close()
    m = re.match(r"password: (?:0x([0-9A-F]+)\s*)?\"(.*)\"$", s)
    if m:
        hexform, stringform = m.groups()
        if hexform:
            return decode_hex(hexform)
        else:
            return stringform


def setpassword(service, account, password):
    cmd = 'security add-generic-password -U -a %s -s %s -p %s' % (account, service, password)
    p = os.popen(cmd)
    s = p.read()
    p.close()

In [23]:
KEYCHAIN = "UFHealth_mom_baby"

In [25]:
proj = redcap.Project('https://redcap.ctsi.ufl.edu/redcap/api/', getpassword(KEYCHAIN, "xinsongdu"))

In [38]:
proj.forms

('baby_clinic_asthma',
 'baby_hospital_asthma',
 'mom_baby_link',
 'mom_medications_rx',
 'baby_hospital_foodallergy',
 'baby_clinic_sebaceous',
 'linked_mom_prental_meds_rx',
 'baby_clinic_ear',
 'mom_prenatal_apt',
 'baby_demography',
 'baby_first_height',
 'mom_demography',
 'baby_clinic_erythema',
 'mom_antibiotics_ip',
 'linked_mom_antibiotics_ip',
 'baby_vaccines',
 'baby_hospital_ear',
 'linked_mom_prenatal_apt',
 'mom_antibiotics_rx',
 'baby_clinic_obesity',
 'baby_wellvisit',
 'baby_clinic_dermatitis',
 'baby_hospital_erythema',
 'baby_hospital_sebaceous',
 'baby_antibiotics_rx',
 'baby_hospital_obesity',
 'linked_mom_demography',
 'baby_antibiotics_ip',
 'baby_clinic_foodallergy',
 'baby_clinic_eczema',
 'baby_clinic_hemangioma',
 'baby_hospital_eczema',
 'baby_first_head_circumference',
 'baby_hospital_hemangioma',
 'baby_hospital_dermatitis',
 'linked_mom_prenatal_abx_rx')

## baby_clinic_asthma

#### RedCap data

In [44]:
data_redcap = proj.export_records(forms=['baby_clinic_asthma'], format='df')

In [53]:
set(data_redcap["redcap_repeat_instrument"])

{'baby_clinic_asthma', 'baby_demography'}

In [51]:
data_redcap[data_redcap.redcap_repeat_instrument=="baby_clinic_asthma"].head()

redcap_repeat_instrument  redcap_repeat_instance  \
part_id   redcap_event_name                                                    
Baby-0001 visit_1_arm_1           baby_clinic_asthma                       1   
Baby-0006 visit_1_arm_1           baby_clinic_asthma                       1   
Baby-0009 visit_1_arm_1           baby_clinic_asthma                       1   
          visit_2_arm_1           baby_clinic_asthma                       2   
          visit_3_arm_1           baby_clinic_asthma                       3   

                            baby_asthma_date  days2_baby_asthma_clinic  \
part_id   redcap_event_name                                              
Baby-0001 visit_1_arm_1           2015-01-15                     465.0   
Baby-0006 visit_1_arm_1           2013-03-12                     650.0   
Baby-0009 visit_1_arm_1           2011-11-09                     161.0   
          visit_2_arm_1           2011-11-10                     162.0   
          visit_3_arm_1           2011-12-15                     197.0   

                            baby_asthma_icd  baby_clinic_asthma_complete  
part_id   redcap_event_name                                               
Baby-0001 visit_1_arm_1               465.9                          0.0  
Baby-0006 visit_1_arm_1               493.9                          0.0  
Baby-0009 visit_1_arm_1               465.9                          0.0  
          visit_2_arm_1               465.9                          0.0  
          visit_3_arm_1               465.9                          0.0

In [56]:
data_redcap[data_redcap.redcap_repeat_instrument=="baby_clinic_asthma"].shape

(13241, 6)

In [58]:
DataFrameSummary(data_redcap[data_redcap.redcap_repeat_instrument=="baby_clinic_asthma"]).summary()

,redcap_repeat_instrument,redcap_repeat_instance,baby_asthma_date,days2_baby_asthma_clinic,baby_asthma_icd,baby_clinic_asthma_complete
count,NaN,13241,NaN,13241,NaN,13241
mean,NaN,5.35443,NaN,584.138,NaN,0
std,NaN,6.53209,NaN,448.073,NaN,0
min,NaN,1,NaN,0,NaN,0
25%,NaN,1,NaN,236,NaN,0
50%,NaN,3,NaN,464,NaN,0
75%,NaN,7,NaN,831,NaN,0
max,NaN,75,NaN,2156,NaN,0
counts,13241,13241,13241,13241,13241,13241
uniques,1,75,1981,1852,36,1


#### Dropbox data

In [41]:
data_dropbox = pd.read_excel(io="../data/Baby-Billing Codes (Clinic).xlsx", sheet_name="Asthma (Clinic)")

In [47]:
data_dropbox.head()

,Baby Id,Service/Charge Date,Asthma ICD9/ICD10
0,Baby-0001,2015-01-15,465.9
1,Baby-0006,2013-03-12,493.90
2,Baby-0009,2011-11-09,465.9
3,Baby-0009,2011-11-10,465.9
4,Baby-0009,2011-12-15,465.9


In [59]:
DataFrameSummary(data_dropbox).summary()

,Baby Id,Service/Charge Date,Asthma ICD9/ICD10
count,13241,13241,13241
unique,3577,1981,36
top,Baby-3337,2014-12-08 00:00:00,465.9
freq,75,28,8009
first,NaN,2011-07-10 00:00:00,NaN
last,NaN,2017-05-31 00:00:00,NaN
counts,13241,13241,13241
uniques,3577,1981,36
missing,0,0,0
missing_perc,0%,0%,0%


## baby_hospital_asthma

#### RedCap data

In [70]:
data_redcap = proj.export_records(forms=['baby_hospital_asthma'], format='df')

In [77]:
data_redcap[data_redcap.redcap_repeat_instrument=="baby_hospital_asthma"].shape

(5293, 6)

In [73]:
DataFrameSummary(data_redcap[data_redcap.redcap_repeat_instrument=="baby_hospital_asthma"]).summary()

,redcap_repeat_instrument,redcap_repeat_instance,baby_asthma_hosp_date,days2_baby_asthma_hosp,baby_asthma_hosp_icd,baby_hospital_asthma_complete
count,NaN,5293,NaN,5293,NaN,5293
mean,NaN,2.9163,NaN,710.693,NaN,0
std,NaN,3.23371,NaN,498.845,NaN,0
min,NaN,1,NaN,0,NaN,0
25%,NaN,1,NaN,308,NaN,0
50%,NaN,2,NaN,595,NaN,0
75%,NaN,3,NaN,1039,NaN,0
max,NaN,29,NaN,2148,NaN,0
counts,5293,5293,5293,5293,5293,5293
uniques,1,29,1663,1642,35,1


#### Dropbox data

In [74]:
data_dropbox = pd.read_excel(io="../data/Baby-Billing Codes (Hospital).xlsx", sheet_name="Asthma")

In [76]:
data_dropbox.shape

(5293, 3)

In [75]:
DataFrameSummary(data_dropbox).summary()

,Baby-Id,Admit Date,ICD9/ICD10 Code
count,5293,5293,5293
unique,2465,1663,35
top,Baby-3337,2016-09-02 00:00:00,4659
freq,29,17,1706
first,NaN,2011-06-19 00:00:00,NaN
last,NaN,2017-05-04 00:00:00,NaN
counts,5293,5293,5293
uniques,2465,1663,35
missing,0,0,0
missing_perc,0%,0%,0%


## mom-baby link

#### RedCap data

In [60]:
data_redcap = proj.export_records(forms=['mom_baby_link'], format='df')

In [69]:
data_redcap.head()

redcap_repeat_instrument  redcap_repeat_instance  \
part_id   redcap_event_name                                                    
Baby-0001 visit_1_arm_1              baby_demography                       1   
          visit_1_arm_1                mom_baby_link                       1   
Baby-0002 visit_1_arm_1              baby_demography                       1   
          visit_1_arm_1                mom_baby_link                       1   
Baby-0003 visit_1_arm_1              baby_demography                       1   

                             part_id2  mom_baby_link_complete  
part_id   redcap_event_name                                    
Baby-0001 visit_1_arm_1           NaN                     NaN  
          visit_1_arm_1           NaN                     0.0  
Baby-0002 visit_1_arm_1           NaN                     NaN  
          visit_1_arm_1           NaN                     0.0  
Baby-0003 visit_1_arm_1           NaN                     NaN

In [62]:
set(data_redcap["redcap_repeat_instrument"])

{'baby_demography', 'mom_baby_link'}

In [63]:
data_redcap[data_redcap.redcap_repeat_instrument=="mom_baby_link"].head()

,,redcap_repeat_instrument,redcap_repeat_instance,part_id2,mom_baby_link_complete
part_id,redcap_event_name,,,,
Baby-0001,visit_1_arm_1,mom_baby_link,1,NaN,0.0
Baby-0002,visit_1_arm_1,mom_baby_link,1,NaN,0.0
Baby-0003,visit_1_arm_1,mom_baby_link,1,NaN,0.0
Baby-0004,visit_1_arm_1,mom_baby_link,1,NaN,0.0
Baby-0005,visit_1_arm_1,mom_baby_link,1,NaN,0.0


In [64]:
data_redcap[data_redcap.redcap_repeat_instrument=="mom_baby_link"].shape

(29798, 4)

In [ ]:
DataFrameSummary(data_redcap[data_redcap.redcap_repeat_instrument=="baby_clinic_asthma"]).summary()

#### Dropbox data

In [65]:
data_dropbox = pd.read_excel(io="../data/Baby-Mom.xlsx", sheet_name="Baby-Mom Link")

In [66]:
data_dropbox.head()

,Baby-Id,Baby Gender,Mom-Id,Mom Age,Payer
0,Baby-0001,MALE,Mom-4388,25,Blue Cross
1,Baby-0002,FEMALE,Mom-1829,21,Medicaid
2,Baby-0003,MALE,Mom-8494,19,Medicaid
3,Baby-0004,FEMALE,Mom-0320,30,Medicaid
4,Baby-0005,FEMALE,Mom-6297,26,Medicaid


In [68]:
data_dropbox.shape

(16649, 5)

In [67]:
DataFrameSummary(data_dropbox).summary()

,Baby-Id,Baby Gender,Mom-Id,Mom Age,Payer
count,NaN,NaN,NaN,16649,NaN
mean,NaN,NaN,NaN,27.6268,NaN
std,NaN,NaN,NaN,5.95228,NaN
min,NaN,NaN,NaN,12,NaN
25%,NaN,NaN,NaN,23,NaN
50%,NaN,NaN,NaN,27,NaN
75%,NaN,NaN,NaN,32,NaN
max,NaN,NaN,NaN,56,NaN
counts,16649,16649,16609,16649,16649
uniques,16649,3,13836,39,11
